<a href="https://colab.research.google.com/github/idieyidot/Python_Learning/blob/master/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:

import pandas as pd
import numpy as np
from google.colab import files


#upload excel sheet from laptop to google colab
uploaded = files.upload() 



Saving coronavirus_sequences_table.xlsx to coronavirus_sequences_table (1).xlsx


In [76]:
# read in all our data
dataset = pd.read_excel("coronavirus_sequences_table.xlsx")
# set seed for reproducibility
np.random.seed(0) 
dataset.sample(5)


,Details,Accession,Release_Date,Species,Genus,Family,Length,Genotype,Genome_Region,Segment,Authors,Publications,Geo_Location,Host,Isolation_Source,Collection_Date,BioSample,GenBank_Title
7057,NaN,DD451150,2007-07-02T00:00:00Z,Severe acute respiratory syndrome-related coro...,Betacoronavirus,Coronaviridae,182,NaN,NaN,NaN,"Vijeisuri,S., Getman,D.K., Linnen,J.M., Nelson...",NaN,NaN,NaN,NaN,NaN,NaN,Compositions and Methods for Determining the P...
3456,NaN,MH741316,2019-01-13T00:00:00Z,Betacoronavirus 1,Betacoronavirus,Coronaviridae,488,NaN,NaN,NaN,"He,Q.F., Guo,Z.J., Yue,H., Tang,C.",NaN,China,Bos grunniens,NaN,2015,NaN,Yak coronavirus isolate XZ8 S protein (S) gene...
7579,NaN,MG772864,2018-03-28T00:00:00Z,Severe acute respiratory syndrome-related coro...,Betacoronavirus,Coronaviridae,430,NaN,NaN,NaN,"Hu,D.",NaN,China,Rhinolophus sinicus,NaN,2016-01,NaN,Bat SARS-like coronavirus isolate bat-SL-CoVZ2...
4284,NaN,JN795147,2012-07-11T00:00:00Z,Betacoronavirus 1,Betacoronavirus,Coronaviridae,1038,NaN,NaN,NaN,"Bidokhti,M.R., Traven,M., Ohlson,A., Baule,C.,...",22750286,Sweden,Bos taurus,feces,2008,NaN,Bovine coronavirus isolate C5_0801 spike glypr...
6963,NaN,AF523850,2002-08-21T00:00:00Z,Betacoronavirus 1,Betacoronavirus,Coronaviridae,292,NaN,NaN,NaN,"Wu,H.Y., Guy,J.S., Yoo,D., Vlasak,R., Urbach,E...",14592769,NaN,NaN,NaN,NaN,NaN,"Equine Coronavirus NC99 3'UTR, partial sequence"


In [77]:

#check how many rows and columns
dataset.shape # (rows, column) respectively.

(7812, 18)

In [78]:
#to check the dimension number for our dataset
dataset.ndim

2

In [79]:
#Check datatypes of the column of our dataset
dataset.dtypes

Details             float64
Accession            object
Release_Date         object
Species              object
Genus                object
Family               object
Length                int64
Genotype             object
Genome_Region       float64
Segment             float64
Authors              object
Publications         object
Geo_Location         object
Host                 object
Isolation_Source     object
Collection_Date      object
BioSample            object
GenBank_Title        object
dtype: object

# 1. Dealing With Missing Values Generally (While doing Feature Selection)

In [80]:
# get the number of missing data points per column
missing_values_count = dataset.isnull().sum()

# look at the # of missing points of first 10 columns
missing_values_count[0:10]

Details          7812
Accession           0
Release_Date        0
Species             0
Genus               0
Family              0
Length              0
Genotype         7804
Genome_Region    7812
Segment          7812
dtype: int64

In [81]:
# how many total missing values do we have?
total_cells = np.product(dataset.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
(total_missing/total_cells) * 100


38.24671445639188

This is for removing all columns that has NAN (missing values)

In [82]:
# pandas drop columns using list of column names (selective drop)
new_dataset = dataset.drop(['Details', 'Genotype', 'Genome_Region','Segment','BioSample', 'Isolation_Source', 'Collection_Date','Publications','Geo_Location', 'Authors'], axis=1)



In [83]:
#saving the dropped column to a new csv file
new_dataset.to_csv("dropped_columns.csv", index=False, encoding ='utf8') 

In [84]:
#check back if it really dropped the column
missing_values_count = new_dataset.isnull().sum()

# look at the # of missing points
missing_values_count[0:18] #checked all column 

#from this, we can see that there's only 9 column left

Accession           0
Release_Date        0
Species             0
Genus               0
Family              0
Length              0
Host             2296
GenBank_Title       0
dtype: int64

In [85]:
# To confirm, let's compare the column number between previous dataset and new dataset
print("Columns in original dataset: %d \n" % dataset.shape[1])
print("Columns in new dataset: %d" % new_dataset.shape[1])

Columns in original dataset: 18 

Columns in new dataset: 8


Feature selection was done by dropping the column that contains missing values as it was not needed while solving the missing values problems. However, there's 1 more column that has missing values, I cannot delete this as it is a crucial information for the host prediction later. 



# 2. Dealing with missing values in Host Column

In [86]:
# replace all NA's with 0
new_dataset2=new_dataset.fillna('null')
new_dataset2.to_csv("no_missing_values.csv", index=False, encoding ='utf8') 

In [87]:
#check back if it really filled the missing values
missing_values_count = new_dataset2.isnull().sum()

# look at the # of missing points
missing_values_count[0:18] #checked all column 

#from this, we can see that there's no more missing values on the host column

Accession        0
Release_Date     0
Species          0
Genus            0
Family           0
Length           0
Host             0
GenBank_Title    0
dtype: int64

# 3. Detect Redundancy and Remove duplicated rows

To find whether a data-set contain duplicate rows or not we can use Pandas DataFrame.duplicated()  either for all columns or for some selected columns. pandas.Dataframe.duplicated() returns a Boolean series denoting duplicate rows

In [88]:
duplicate_rows_df = new_dataset2[new_dataset2.duplicated()]
print ("Number of duplicate rows: ", duplicate_rows_df.shape)

Number of duplicate rows:  (0, 8)


It is shown (row, column) respectively means 0 duplicated rows in 8 columns. 

# 4.Scaling & Normalization

In [89]:
# for Box-Cox Transformation
from scipy import stats

# for min_max scaling
from mlxtend.preprocessing import minmax_scaling

# plotting modules
import seaborn as sns
import matplotlib.pyplot as plt

# set seed for reproducibility
np.random.seed(0)

**Normalization** 
The point of normalization is to change your observations so that they can be described as a normal distribution.

**Scaling** his means that you're transforming your data so that it fits within a specific scale, like 0-100 or 0-1. 

However, a dataset should be split before doing normalization to avoid biased evaluation

In [90]:

# get the index of all positive pledges (Box-Cox only takes postive values)
#index_Length = dataset3.Length > 0

# get only positive pledges (using their indexes)
#positive_Lengths = dataset3.Length.loc[index_Length]

# normalize the pledges (w/ Box-Cox)
#normalized_Lengths = stats.boxcox(positive_Lengths)[0]

# plot both together to compare
#fig, ax=plt.subplots(1,2)
#sns.distplot(positive_Lengths, ax=ax[0])
#ax[0].set_title("Original Data")
#sns.distplot(normalized_Lengths, ax=ax[1])
#ax[1].set_title("Normalized data")
#model._normalization_mean.numpy()

# 5. Splitting datasets into three sets

To make it easier to get sequences from Batch Entrez

In [95]:
new_dataset2.head()

,Accession,Release_Date,Species,Genus,Family,Length,Host,GenBank_Title
0,AB277133,2007-03-24T00:00:00Z,Betacoronavirus 1,Betacoronavirus,Coronaviridae,411,null,Bovine coronavirus S1 gene for spike protein s...
1,AB277120,2007-03-24T00:00:00Z,Betacoronavirus 1,Betacoronavirus,Coronaviridae,411,null,Bovine coronavirus S1 gene for spike protein s...
2,AB450865,2008-11-11T00:00:00Z,Betacoronavirus 1,Betacoronavirus,Coronaviridae,411,null,Bovine coronavirus S1 gene for spike protein s...
3,AB450878,2008-11-11T00:00:00Z,Betacoronavirus 1,Betacoronavirus,Coronaviridae,411,null,Bovine coronavirus S1 gene for spike protein s...
4,AB450881,2008-11-11T00:00:00Z,Betacoronavirus 1,Betacoronavirus,Coronaviridae,411,null,Bovine coronavirus S1 gene for spike protein s...


**Method:** Splitting Pandas Dataframe by row index (edited, I will just categorize the data using the host then split them by host. However code below will remain here as reference

In [92]:
# splitting dataframe by row index
#df_1 = new_dataset2.iloc[:2604]
#df_2 = new_dataset2.iloc[2604:5209]
#df_3 = new_dataset2.iloc[5209:]
#print("Shape of new dataframes - {} , {}, {}".format(df_1.shape, df_2.shape, df_3.shape))

In [93]:
#saving the three datasets into 3 seperate csv files

#df_1.to_csv("cleaned1_2604rows.csv", index=False, encoding ='utf8') 
#df_2.to_csv("cleaned2_2605rows.csv", index=False, encoding ='utf8') 
#df_3.to_csv("cleaned3_2603rows.csv", index=False, encoding ='utf8') 

In [154]:
#sorting the dataset by the column host
s = new_dataset2.sort_values('Host', ascending=False)

#count occurance of host names in dataframe
pd.set_option("max_columns",2)
pd.set_option ("Max_rows", None)
s.Host.value_counts()

#splitting the dataset by the column host

Homo sapiens                  2358
null                          2296
Bos taurus                     614
Camelus dromedarius            482
Eidolon helvum                 341
Bos grunniens                  139
Rhinolophus sinicus            130
Camelus                        127
Chiroptera                     125
Rhinolophus ferrumequinum       82
Rousettus leschenaultii         76
Mus musculus                    73
Sus scrofa                      53
Rousettus sp.                   52
Micropteropus pusillus          47
Eonycteris spelaea              44
Bovidae                         37
Neoromicia capensis             37
Equus caballus                  33
Tylonycteris pachypus           31
Erinaceus europaeus             30
Epomops franqueti               30
Rhinolophus monoceros           28
Vicugna pacos                   24
Oryctolagus cuniculus           22
Canis lupus familiaris          19
Paradoxurus hermaphroditus      18
Epomophorus gambianus           16
Megaloglossus woerma